In [156]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib
import scipy
from itertools import combinations

In [227]:
def addLayer(circ, thetas):
    for i in range(circ.num_qubits):
        circ.rz(thetas[i],i)
        
    if circ.num_qubits > 1:
        for c, t in combinations(range(circ.num_qubits),2):
            circ.cz(c,t)
        
    for i in range(circ.num_qubits):
        circ.rx(thetas[i],i)
        
def getCircuit(numQubits, numLayers):
    thetas = [[Parameter(f'θ[{i},{j+1}]') for j in range(numQubits)] for i in range(numLayers)]
    thetas = np.array(thetas)
    
    circ = QuantumCircuit(numQubits)
    for i in range(numLayers):
        addLayer(circ, thetas[i])
        circ.barrier()
        
    return circ, thetas
        
def generatePsi(numQubits, numLayers, thetaValues):
    circ, parameters = getCircuit(numQubits, numLayers)
    
    parameter_binds={}
    n,m = parameters.shape
    for i in range(n):
        for j in range(m):
            parameter_binds[parameters[i][j]] = thetaValues[i][j]
    
    simulator = Aer.get_backend('statevector_simulator')
    return execute(circ, simulator, parameter_binds=[parameter_binds], shots=1024).result().get_statevector(circ)

def AddOrangeBlock(circ, theta1, theta2, theta3, theta4):
    circ.rz(theta1,0)
    circ.rz(theta2,1)
    circ.rz(theta3,2)
    circ.rz(theta4,3)
    
    circ.cz(0,1)
    circ.cz(0,2)
    circ.cz(0,3)
    circ.cz(1,2)
    circ.cz(1,3)
    circ.cz(2,3)
    
def AddGreenBlock(circ, theta1, theta2, theta3, theta4):
    circ.rx(theta1,0)
    circ.rx(theta2,1)
    circ.rx(theta3,2)
    circ.rx(theta4,3)
    
def AddLayer(circ, theta1, theta2, theta3, theta4):
    AddOrangeBlock(circ, theta1, theta2, theta3, theta4)
    AddGreenBlock(circ, theta1, theta2, theta3, theta4)
    
def generateRandomState(size):
    vector = np.random.random(size) + np.random.random(size) * 1j
    norm = np.linalg.norm(vector)
    return vector/norm

def generatePsiOneLayer(theta1, theta2, theta3, theta4):
    circuit = QuantumCircuit(reg)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    AddLayer(circuit, theta1, theta2, theta3, theta4)
    simulator = Aer.get_backend('statevector_simulator')
    return execute(circuit, simulator, shots=1024).result().get_statevector(circuit)


In [241]:
fi = generateRandomState(16)

In [310]:
def cost(flattenedThetas):
    numQubits = 4
    numLayers = 4
    thetas = flattenedThetas.reshape(numLayers, numQubits)
    psi = generatePsi(numQubits,numLayers,thetas)
    return np.linalg.norm(psi-fi)

In [315]:
numQubits = 4
numLayers = 4
start = np.random.uniform(low=0, high=2*np.pi, size=(numQubits*numLayers,))
res = scipy.optimize.minimize(cost, start, method='COBYLA')
res

     fun: 0.31567457155921386
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 1000
  status: 2
 success: False
       x: array([5.93467471, 5.88568726, 6.28145433, 4.64856485, 1.6553404 ,
       4.64556344, 1.56102026, 4.92259821, 3.49523539, 0.16557739,
       0.04351441, 4.02475405, 4.79182638, 4.79805291, 1.43810092,
       0.82368993])

In [137]:
# def cost(theta):
#     psi = generatePsiOneLayer(theta[0], theta[1], theta[2], theta[3])
#     return np.linalg.norm(psi-fi)
    
# start = np.random.uniform(low=0, high=2*np.pi, size=(4,))
# res = scipy.optimize.minimize(cost, start, method='Nelder-Mead')
# res

 final_simplex: (array([[4.93823265, 5.62600736, 4.08638887, 2.93643434],
       [4.9382118 , 5.62610462, 4.08632564, 2.93648374],
       [4.93823723, 5.62600619, 4.08631111, 2.93646639],
       [4.93820811, 5.62601928, 4.0863378 , 2.93649545],
       [4.93813918, 5.62609026, 4.08631114, 2.93652383]]), array([0.91815234, 0.91815234, 0.91815234, 0.91815235, 0.91815235]))
           fun: 0.9181523397796332
       message: 'Optimization terminated successfully.'
          nfev: 139
           nit: 81
        status: 0
       success: True
             x: array([4.93823265, 5.62600736, 4.08638887, 2.93643434])

In [133]:
cost([2.52458136, 2.18068981, 4.0635977 , 1.54155779])


0.8990668053466107

In [49]:
fi = generateRandomState(16)

In [66]:
np.linalg.norm(psi-fi)

1.365507537993799

In [237]:
c, t = getCircuit(3,2)
c.draw()

┌────────────┐      ┌────────────┐               ░ ┌────────────┐      »
q_0: ┤ RZ(θ[0,1]) ├─■──■─┤ RX(θ[0,1]) ├───────────────░─┤ RZ(θ[1,1]) ├─■──■─»
     ├────────────┤ │  │ └────────────┘┌────────────┐ ░ ├────────────┤ │  │ »
q_1: ┤ RZ(θ[0,2]) ├─■──┼───────■───────┤ RX(θ[0,2]) ├─░─┤ RZ(θ[1,2]) ├─■──┼─»
     ├────────────┤    │       │       ├────────────┤ ░ ├────────────┤    │ »
q_2: ┤ RZ(θ[0,3]) ├────■───────■───────┤ RX(θ[0,3]) ├─░─┤ RZ(θ[1,3]) ├────■─»
     └────────────┘                    └────────────┘ ░ └────────────┘      »
«     ┌────────────┐               ░ 
«q_0: ┤ RX(θ[1,1]) ├───────────────░─
«     └────────────┘┌────────────┐ ░ 
«q_1: ──────■───────┤ RX(θ[1,2]) ├─░─
«           │       ├────────────┤ ░ 
«q_2: ──────■───────┤ RX(θ[1,3]) ├─░─
«                   └────────────┘ ░

In [284]:
psi = generatePsi(3,2,[[np.pi, np.pi/2, np.pi/3],[np.pi/3, np.pi/4, np.pi]])
np.linalg.norm(fi)

1.0

In [308]:
p = generatePsi(4,3,[[4.5912527 ,  1.58697777,  4.88585462,  4.49225959],  [6.08829631,
        6.63230784,  3.63558275,  6.95567655],  [2.20537385, -1.53319168,
        5.28154685,  1.16467469]])
np.linalg.norm(p-fi)

0.36941506072438274